# This notebook was used to calculate Cross Validation scores for models

## Additional data sources

https://stat.gov.pl/statystyka-regionalna/rankingi-statystyczne/ludnosc-wedlug-wojewodztw/

https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce

## Base configuration

In [1]:
import helper as h

import pandas as pd
import numpy as np
import time
import re

from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error as mae

np.random.seed(2018)
pd.set_option('display.max_columns', None)

price_shift = 50000


## Data

In [2]:
train = pd.read_hdf('../input/train_property.h5')
test = pd.read_hdf('../input/test_property.h5') 

In [3]:
stat_miasta = pd.read_csv('../externalData/statystyki-miasta-wiki.csv')
stat_woj = pd.read_csv('../externalData/statystyki-woj.csv')

## FEATURE ENGINEERING

In [4]:
######################################################
#
# Helper functions
#

def norm_date(value):
    if value is None: return value
    
    months_to_digit = {
        'stycznia': 1,
        'lutego': 2,
        'marca': 3,
        'kwietnia': 4,
        'maja': 5,
        'czerwca': 6,
        'lipca': 7,
        'sierpnia': 8,
        'września': 9,
        'października': 10,
        'listopada': 11,
        'grudnia': 12
    }
    values = value.split(' ')

    day   = int(values[0]) if len(values) == 3 else None
    month = values[-2].lower()
    year  = int(values[-1])

    month = months_to_digit[month]

    return (year*100 + month)


def map_city(x):
    if (len(x) == 2):
        if x[1] in miasta_ll_dict:
            return x[1]
        else:
            return 'unknown'
    if (len(x)>2):
        if x[2] in miasta_ll_dict:
            return x[2]
        elif x[1] in miasta_ll_dict:
            return x[1]
        else: 
            return 'unknown'
    
    return 'unknown'

def get_visit_ads(x):
    if 'visit_ads' in x:
        return np.log( int(x['visit_ads']) + 10 )
    return -1

#
# extract price from the 'text' field
#
def extract_largest_value(x):
    formated = x.lower().replace('m2','').replace(' ','')
    digits = re.split("\D", formated)
    max_value = 0
    if x.lower().find('cena') > 0: # only if there is 'cena' in text
        for i in digits:
            if i.isdigit():
                if len(i) < 7: # skip telephone numbers and other identifiers
                    if int(i) < 1000000: # skip too high values
                        max_value = max(int(i),max_value)

        if max_value > 100000: # skip too small values
            return(max_value)
    
    return 0

##########################################################################
#
# Categorize function
#

def categorize_feature(df, feat, feat_cat, indexers, del_feat=True, average_func=np.median, unknown_cat_name='unknown', unknown_cat_val=-1):
    if feat in indexers:
        return categorize_feature_for_test(df, feat, feat_cat, indexers[feat], del_feat=del_feat, unknown_cat_name=unknown_cat_name, unknown_cat_val=unknown_cat_val)
    
    categories_map = categorize_feature_for_train(df, feat, feat_cat, indexers, del_feat=del_feat, unknown_cat_name=unknown_cat_name)
    
def categorize_feature_for_test(df, feat, feat_cat, categories_map, del_feat=True, unknown_cat_name='unknown', unknown_cat_val=-1):
    df[feat_cat] = [categories_map[x] if x in categories_map 
                    else categories_map[unknown_cat_name] if unknown_cat_name in categories_map 
                    else unknown_cat_val for x in df[feat]]
    
    if del_feat:
        del df[feat]
    
def categorize_feature_for_train(df, feat, feat_cat, indexers, del_feat=True, unknown_cat_name='unknown'):
    df.loc[df[feat].isnull(), feat] = unknown_cat_name

    unique_categories = sorted(list(set(df[feat])),key=str)
    
    categories_map = {}

    for i, (cat) in enumerate(unique_categories):
        categories_map[cat] = i

    for cat, ind in categories_map.items():
        df.loc[df[feat] == cat, feat_cat] = ind

    indexers[feat] = categories_map
    
    if del_feat:
        del df[feat]

##########################################################################
#
# Data preparation
#

def perform_engineering(df, train_indexers=None):
    indexers = train_indexers if train_indexers != None else {}
    
    df['area_float'] = df['area'].map(lambda x: float(re.sub('[^0-9\,\.]','', x).replace(',', '.')))
    df['area_fixed'] = df['area_float'].map(lambda x: x if x > 0 else 60)
    df['floor_int'] = df['floor'].map({'parter':0, '1':1, '2':2, '3':3, -1:-1, '4':4, 
                                       '7':7, '5':5, '10':10, '8':8, '6':6, '9':9,'> 10':11, 
                                       'poddasze':12, 'suterena':-2})
    df['floors_in_int'] = df['floors_in_building'].map(lambda x: 
                                                       int(re.sub('[^0-9\,\.]','', x)) if x != -1 else -1 )
    df['rok_budowy_int'] = df['rok budowy'].map(lambda x: int(re.sub('[^0-9\,\.]','', x)) if x != -1 else -1 )
    df['czynsz_float'] = df['czynsz'].map(lambda x: 
                                          float(re.sub('[^0-9\,\.]','', x).replace(',', '.')) if x != -1 else x )
    
    #
    # LOCATION
    #
    df['location_v'] = df['location'].map(lambda x: x[0])
    df['location_c'] = df['location'].map(lambda x: x[0] + ", " + x[1])
    df['location_cc'] = df['location'].map(
            lambda x: x[0] + ", " + x[1] + ", " + x[2] if len(x)>2 else x[0] + ", " + x[1])  
    df['location_powiat'] = df['location'].map(lambda x: x[1] if x[1][-1]=='i' else 'unknown')
    df['location_miasto'] = df['location'].map(map_city)
    
    df['miasto_ludnosc'] = df['location_miasto'].map(miasta_ll_dict)
    df['miasto_gestosc'] = df['location_miasto'].map(miasta_gz_dict)
    df['miasto_powierzchnia'] = df['location_miasto'].map(miasta_pow_dict)
    df['wojewodztwo_ludnosc'] = df['location_v'].map(wojewodztwa_ll)
    
    categorize_feature(df, 'location_v', 'location_v_cat', indexers=indexers)
    categorize_feature(df, 'location_c', 'location_c_cat', indexers=indexers)
    categorize_feature(df, 'location_cc', 'location_cc_cat', indexers=indexers)

    categorize_feature(df, 'location_powiat', 'location_powiat_cat', indexers=indexers)
    categorize_feature(df, 'location_miasto', 'location_miasto_cat', indexers=indexers)
    
    #
    # Stats
    #
    
    df['stats_created_at'] = df['stats'].map(lambda x: x['created_at'])
    df['stats_updated_at'] = df['stats'].map(lambda x: x['updated_at'])
    df['stats_visit_ads'] = df['stats'].map(get_visit_ads)
    
    categorize_feature(df, 'stats_created_at', 'stats_created_at_cat', indexers=indexers)
    categorize_feature(df, 'stats_updated_at', 'stats_updated_at_cat', indexers=indexers)
    
    #
    # text field - price extraction from the text
    #
    
    df['largest_value'] = df['text'].map(extract_largest_value)
    
    #
    # categorization
    #
    categorize_feature(df, 'materiał budynku', 'material_cat', indexers=indexers)
    categorize_feature(df, 'okna', 'okna_cat', indexers=indexers)
    categorize_feature(df, 'stan wykończenia', 'stan_wyk_cat', indexers=indexers)
    categorize_feature(df, 'rodzaj zabudowy', 'rodzaj_zabudowy_cat', indexers=indexers)
    categorize_feature(df, 'ogrzewanie', 'ogrzewanie_cat', indexers=indexers)
    categorize_feature(df, 'forma własności', 'forma_wlasnosci_cat', indexers=indexers)
    
    #
    # cleanup
    #
    columns_to_remove = ['area', 'location', 'data rozpoczęcia', 'stan inwestycji', 'liczba kondygnacji',
                         'floor','floors_in_building','rok budowy', 'czynsz', 'dostępne od',
                         'garaż', 'tarasy', 'ochrona', 'stats', 'text', 'rolety antywłamaniowe', 
                         'kuchenka', 'klimatyzacja', 'plan zagospodarowania:',
                         'telefon', 'telewizja kablowa', 'pom. użytkowe', 'pralka', 'piekarnik',
                         'lodówka', 'ogródek', 'drzwi / okna antywłamaniowe'
                        ]
    for col_to_remove in columns_to_remove:
        if col_to_remove in df: del df[col_to_remove]

    return df, indexers

def price_engineering(df):
    df['lv_mean'] = df['location_v_cat'].map(dict_mean_price_by_LV)
    df['lv_median'] = df['location_v_cat'].map(dict_median_price_by_LV)
    df['lc_mean'] = df['location_c_cat'].map(dict_mean_price_by_LC)
    df['lc_median'] = df['location_c_cat'].map(dict_median_price_by_LC)
    df['lc_mean'] = df['lc_mean'].fillna(df['lv_mean'])
    df['lc_median'] = df['lc_median'].fillna(df['lv_median'])
    df['lcc_mean'] = df['location_cc_cat'].map(dict_mean_price_by_LCC)
    df['lcc_median'] = df['location_cc_cat'].map(dict_median_price_by_LCC)
    df['lcc_mean'] = df['lcc_mean'].fillna(df['lc_mean'])
    df['lcc_median'] = df['lcc_median'].fillna(df['lc_median'])

    df['lv_mean_price_calculated'] = df['lv_mean'] * df['area_fixed']
    df['lv_median_price_calculated'] = df['lv_median'] * df['area_fixed']
    df['lc_mean_price_calculated'] = df['lc_mean'] * df['area_fixed']
    df['lc_median_price_calculated'] = df['lc_median'] * df['area_fixed']
    df['lcc_mean_price_calculated'] = df['lcc_mean'] * df['area_fixed']
    df['lcc_median_price_calculated'] = df['lcc_median'] * df['area_fixed']   
    
    #
    # cleanup
    #
    columns_to_remove = ['price_per_meter']
    for col_to_remove in columns_to_remove:
        if col_to_remove in df: del df[col_to_remove]

#########################################
#            
# statistics
#
miasta_gz_dict = stat_miasta.groupby('Miasto').agg(np.median)['Gęstość zaludnienia'].to_dict()
miasta_ll_dict = stat_miasta.groupby('Miasto').agg(np.median)['Liczba ludności'].to_dict()
miasta_pow_dict = stat_miasta.groupby('Miasto').agg(np.median)['Powierzchnia'].to_dict()
wojewodztwa_ll = stat_woj.groupby('Województwo').agg(np.median)['Ogółem'].to_dict()
            
## fill NA
train = train.fillna(-1)
test = test.fillna(-1)

## prepare data
%time train, indexers = perform_engineering(train)
%time test, dummy = perform_engineering(test, indexers)

#########################################
#
# train specific (only for train)
#
train['price_per_meter'] = train['price'] / train['area_fixed']

dict_mean_price_by_LV = train.groupby('location_v_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LV = train.groupby('location_v_cat').agg(np.median)['price_per_meter'].to_dict()
dict_mean_price_by_LC = train.groupby('location_c_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LC = train.groupby('location_c_cat').agg(np.median)['price_per_meter'].to_dict()
dict_mean_price_by_LCC = train.groupby('location_cc_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LCC = train.groupby('location_cc_cat').agg(np.median)['price_per_meter'].to_dict()

## prepare price related data
%time price_engineering(train)
%time price_engineering(test)

## fill NA
train = train.fillna(-1)
test = test.fillna(-1)


CPU times: user 1min 13s, sys: 165 ms, total: 1min 13s
Wall time: 1min 13s
CPU times: user 14 s, sys: 19.6 ms, total: 14 s
Wall time: 14 s
CPU times: user 95.6 ms, sys: 4.26 ms, total: 99.9 ms
Wall time: 93.5 ms
CPU times: user 103 ms, sys: 250 µs, total: 104 ms
Wall time: 95.9 ms


## Features Selection

In [5]:
###########################################################################
#
# PREPARE FEATURES
#

black_list = ['price', 'id']

bool_features = train.select_dtypes(include=[np.bool]).columns.values.tolist()

cat_feats = [feat for feat in train.columns if 'cat' in feat]
cat_feats = cat_feats + ['rooms', 'is_private', 'floor_int', 'floors_in_int']

numeric_features = train.select_dtypes(include=[np.float64, np.int64, np.int8]).columns.values
numeric_features = [feat for feat in numeric_features if feat not in (black_list + cat_feats) ]

feats = bool_features + numeric_features + cat_feats

feats = [feat for feat in feats if feat not in (black_list)]

X = train[ feats ].values
y = train[ 'price' ].values

print("Selected features: ", feats)

Selected features:  ['monitoring / ochrona', 'taras', 'balkon', 'dwupoziomowe', 'system alarmowy', 'teren zamknięty', 'internet', 'winda', 'piwnica', 'telewizor', 'domofon / wideofon', 'oddzielna kuchnia', 'zmywarka', 'garaż/miejsce parkingowe', 'meble', 'is_primary_market', 'area_float', 'area_fixed', 'rok_budowy_int', 'czynsz_float', 'miasto_ludnosc', 'miasto_gestosc', 'miasto_powierzchnia', 'wojewodztwo_ludnosc', 'stats_visit_ads', 'largest_value', 'lv_mean', 'lv_median', 'lc_mean', 'lc_median', 'lcc_mean', 'lcc_median', 'lv_mean_price_calculated', 'lv_median_price_calculated', 'lc_mean_price_calculated', 'lc_median_price_calculated', 'lcc_mean_price_calculated', 'lcc_median_price_calculated', 'location_v_cat', 'location_c_cat', 'location_cc_cat', 'location_powiat_cat', 'location_miasto_cat', 'stats_created_at_cat', 'stats_updated_at_cat', 'material_cat', 'okna_cat', 'stan_wyk_cat', 'rodzaj_zabudowy_cat', 'ogrzewanie_cat', 'forma_wlasnosci_cat', 'rooms', 'is_private', 'floor_int', '

# XGB

### Helper functions and configuration

In [6]:
global fair_constant 

def fair_obj(y_true, y_pred):
    x = y_pred - y_true
    
    global fair_constant 

    den = abs(x) + fair_constant
    
    grad = fair_constant * x / den
    hess = (fair_constant * fair_constant) / (den * den)
    
    return grad, hess

xgb_params_1 = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 8, 
    'n_estimators': 2500, 
    'learning_rate': 0.04, 
    'min_child_weight': 8, 
    'random_state': 4096
}

xgb_params_2 = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 14, 
    'n_estimators': 420, 
    'learning_rate': 0.056693922378212164, 
    'min_child_weight': 8, 
    'random_state': 2018
}

xgb_params_s = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 8, 
    'n_estimators': 200, 
    'learning_rate': 0.05, 
    'min_child_weight': 8, 
    'random_state': 2018
}

shift = price_shift
fair_constant = 13259.556042072305
y_log = np.log(y + shift)

### Model training

In [7]:
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score as R2 
from sklearn.model_selection import KFold

def modPrice(x,limit):
    if x['largest_value'] == 0:
        return x['XGB_s_results']
    if (abs(x['XGB_s_results'] - x['largest_value']) < limit):
        return x['largest_value']
    
    return x['price']

y_log = np.log(y + shift)
    
cv = KFold(n_splits=6, shuffle=True, random_state=2018)

scores = {
    'x1_R2' : [],
    'x1_MAE' : [],
    'x2_R2' : [],
    'x2_MAE' : [],
    'x_s_R2' : [],
    'x_s_MAE' : [],
    'x_f_R2' : [],
    'x_f_MAE' : []
}

def perform_scoring(m, y_true, y_log_pred):
    y_log_pred[ y_log_pred < 0 ] = 1e-6

    y_pred = np.exp( y_log_pred ) - shift

    score = R2(y_true, y_pred)
    print(m + " R2: ", score)
    scores[m + '_R2'].append(score)
    
    score = mae(y[test_idx], y_pred)
    print(m + " MAE: ", score)
    scores[m + '_MAE'].append(score)

fold = 0
for train_idx, test_idx in cv.split(X):
    fold += 1
    print("fold: ", fold)
    
    # first model
    model = XGBRegressor(**xgb_params_1)
    model.fit(X[train_idx], y_log[train_idx])
    y_log_pred = model.predict(X[test_idx])
    perform_scoring('x1', y[test_idx], y_log_pred)
    
    train['XGB_1_results'] = model.predict(X)
    
    # second model
    model = XGBRegressor(**xgb_params_2)
    model.fit(X[train_idx], y_log[train_idx])
    y_log_pred = model.predict(X[test_idx])
    perform_scoring('x2', y[test_idx], y_log_pred)
    
    train['XGB_2_results'] = model.predict(X)
    
    # stacked
    stacked_feats = feats + ['XGB_1_results', 'XGB_2_results']

    X_stacked = train[ stacked_feats ].values
    y_stacked = train[ 'price' ].values

    y_stacked_log = np.log(y_stacked + shift)

    model = XGBRegressor(**xgb_params_s)
    model.fit(X_stacked[train_idx], y_stacked_log[train_idx])
    
    y_log_pred = model.predict(X_stacked[test_idx])
    perform_scoring('x_s', y_stacked[test_idx], y_log_pred)
    
    y_log_pred = model.predict(X_stacked)
    y_log_pred[ y_log_pred < 0 ] = 1e-6
    y_pred = np.exp( y_log_pred ) - shift
    train['XGB_s_results'] = y_pred
    
    #function
    train['XGB_s_results_mod'] = train.apply(lambda x: modPrice(x,225000), axis=1 )
    y_pred = train['XGB_s_results_mod'].values
    
    score = R2(y[test_idx],y_pred[test_idx])
    print("x_f R2: ", score)
    scores['x_f_R2'].append(score)
    score = mae(y[test_idx],y_pred[test_idx])
    print("x_f MAE: ", score)
    scores['x_f_MAE'].append(score)
    
for s in scores:
    print(s, np.mean(scores[s]), np.std(scores[s]))


fold:  1
x1 R2:  0.7247714132123868
x1 MAE:  47161.945
x2 R2:  0.7094368241853568
x2 MAE:  48387.176
x_s R2:  0.7218475514368436
x_s MAE:  46858.848
x_f R2:  0.7229471094273049
x_f MAE:  45127.67615881888
fold:  2
x1 R2:  0.7831547528623674
x1 MAE:  45913.043
x2 R2:  0.7701184785436026
x2 MAE:  46379.004
x_s R2:  0.7303429703061826
x_s MAE:  45872.496
x_f R2:  0.7317795671628913
x_f MAE:  44085.40977913655
fold:  3
x1 R2:  0.8444629860599884
x1 MAE:  45038.836
x2 R2:  0.832386065000086
x2 MAE:  45577.066
x_s R2:  0.8444881999416802
x_s MAE:  44748.945
x_f R2:  0.8462224655582088
x_f MAE:  42900.029653821955
fold:  4
x1 R2:  0.7695790347437557
x1 MAE:  46504.695
x2 R2:  0.7760659184212069
x2 MAE:  46376.688
x_s R2:  0.7736475834946668
x_s MAE:  45973.773
x_f R2:  0.7745988069745299
x_f MAE:  43956.976237819435
fold:  5
x1 R2:  0.841226168567879
x1 MAE:  44010.645
x2 R2:  0.8330127716367112
x2 MAE:  44638.05
x_s R2:  0.8415169769399039
x_s MAE:  43672.62
x_f R2:  0.8420537544378636
x_f M